In [1]:
import numpy as np
import pandas as pd
from scipy.stats import laplace
import math

data = pd.read_csv('dados/adult.csv')

## Consultas

In [2]:
# Q1: Média do atributo age
def consulta1(data):
    return data.age.mean()
    
# Q2: Somatório do atributo capital-gain
def consulta2(data):
    return data.capitalgain.sum()

# Q3: Média do atributo hours-per-week
def consulta3(data):
    return data.hoursperweek.mean()

# Q4: Quantidade de pessoas com salário maior que 50K
def consulta4(data):
    salario_alto = 0
    for i in range(len(data.index)):
        if data.loc[i, 'class'] == '>50K':
            salario_alto += 1
#     print("Pessoas com salário alto: " + str(salario_alto))
#     print()
    return salario_alto

## Resposta com ruído

In [3]:
def resposta_ruido(data, consulta, budget, sens):
    res = consulta(data)
    ruido = laplace.rvs(loc=0, scale=sens/budget)
    print("Resposta com ruído: " + str(ruido + res))
    return ruido + res

## Obtendo sensibilidades

In [4]:
# sensibilidade da primeira consulta
def sense_q1(data):
    aux = data.copy()
    max_val = data.age.max()
    for i in range(len(aux.index)):
        if(aux.loc[i, 'age'] == max_val):
            aux = aux.drop(i)
            break
    return abs(max_val - consulta1(aux))


# sensibilidade da segunda consulta
def sense_q2(data):
    aux = data.copy()
    max_val = data.capitalgain.max()
    for i in range(len(aux.index)):
        if(aux.loc[i, 'capitalgain'] == max_val):
            aux = aux.drop(i)
            break
    return abs(max_val - consulta2(aux))


# sensibilidade da terceira consulta
def sense_q3(data):
    aux = data.copy()
    max_val = data.hoursperweek.max()
    for i in range(len(aux.index)):
        if(aux.loc[i, 'hoursperweek'] == max_val):
            aux = aux.drop(i)
            break
    return abs(max_val - consulta3(aux))

## a quarta consulta sempre terá sensibilidade 1

## Gerando CSV

In [5]:
def gerar_csv(budgets, res, sens):
    resultado = []
    for i in range(0, len(budgets)):
        reg = [budgets[i]]
        
        for j in range(0, len(res[i])):
            reg.append(res[i][j])
            
        for k in range(0, len(sens)):
            reg.append(sens[k])
            
        resultado.append(reg)
    resultado = pd.DataFrame(resultado, columns=['budget', 'result_q1', 'result_q2', 'result_q3', 'result_q4', 'sens_q1', 'sens_q2', 'sens_q3', 'sens_q4'])
    resultado.to_csv('saida/result.csv', index=False)

## Execução

In [6]:
budgets = [0.1, 1, 10]
consultas = [consulta1, consulta2, consulta3, consulta4]
sensibilidades = [sense_q1(data), sense_q2(data), sense_q3(data), 1]

# Função principal
def dp_laplace(data, budgets, consultas, sens):
    print("Budgets: " + str(budgets))
    print("Consultas: " + str(consultas))
    print("Sensibilidades: " + str(sens))
    resultado = []
    for b in budgets:
        res_budget = []
        for i in range(0, len(consultas)):
            # adicionando a resposta com ruído às respostas de cada budget
            res_budget.append(resposta_ruido(data, consultas[i], b / len(consultas), sens[i]))
        resultado.append(res_budget)
    gerar_csv(budgets, resultado, sens)

In [7]:
dp_laplace(data, budgets, consultas, sensibilidades)

Chamando a função principal!
Budgets: [0.1, 1, 10]
Consultas: [<function consulta1 at 0x0A128270>, <function consulta2 at 0x0A116660>, <function consulta3 at 0x0A116348>, <function consulta4 at 0x0A116FA8>]
Sensibilidades: [2.2289674658586023, 9776, 2.0493437890297086, 1]
Resposta com ruído: -89.12973900394579
Resposta com ruído: -353720.9090440271
Resposta com ruído: 58.48616852744399
Resposta com ruído: 11673.460913205892
Resposta com ruído: -2.113048190302327
Resposta com ruído: -62459.675011628424
Resposta com ruído: 14.424712488244687
Resposta com ruído: 11684.931533232217
Resposta com ruído: 1.9074482681953324
Resposta com ruído: 7911.209127249522
Resposta com ruído: 1.8191367767727513
Resposta com ruído: 11687.873069125002
